In [3]:
import torch
import pandas as pd 
import numpy as np
import os 
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
from torch import optim

In [4]:
data_df = pd.read_csv('data/fiducials_signal_01_0001.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data/fiducials_signal_01_0001.csv'

In [ ]:
data_df

In [ ]:
d_list = []
nan_index = []
counter = 0
for dir in os.listdir('data'):
    if dir == '.ipynb_checkpoints' :
        continue
    df = pd.read_csv('data/'+dir)
    counter+=1
    if df.isnull().values.any():
        print(counter-1)
        nan_index.append(counter-1)
        print(dir)
        continue
    d_list.append(df.values.tolist()[0:9])

In [ ]:
label_df = pd.read_csv('labels_final.csv')
label_df.head()

In [ ]:
for i in nan_index:
    label_df.drop(i, inplace=True)

In [ ]:
label_df.head()

In [ ]:
Glucose = label_df['Glucose'].to_numpy()
Glucose_level = label_df['Glucose_Level'].to_numpy()

In [ ]:
X_train, sub_x, y_train, sub_y = train_test_split(d_list, Glucose, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(sub_x, sub_y, test_size = 1/3, random_state=42)

In [ ]:
x_train_t = torch.tensor(X_train).float()
x_val_t = torch.tensor(X_val).float()
x_test_t = torch.tensor(X_test).float()
y_train_t = torch.tensor(y_train).float()
y_val_t = torch.tensor(y_val).float()

In [ ]:
#conver data to tensors
x_train_t = torch.tensor(X_train).float()
x_val_t = torch.tensor(X_val).float()
x_test_t = torch.tensor(X_test).float()
y_train_t = torch.tensor(y_train).float()
y_val_t = torch.tensor(y_val).float()
#dataset 
train_tensor = TensorDataset(x_train_t,y_train_t)
valid_tensor = TensorDataset(x_val_t,y_val_t)
# dataloader
dataloaders = dict()
dataloaders["train"] = DataLoader(train_tensor, batch_size=23, shuffle=True)
dataloaders["val"] = DataLoader(valid_tensor, batch_size=7)
dataloaders["test"] = DataLoader(x_test_t, batch_size=7)

In [ ]:
dataloaders["test2"] = DataLoader(x_train_t, batch_size=7)

In [ ]:
# Definition of a RNN Model class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, seq_len, dropout=0.025, output_size=1):
        super(RNN, self).__init__()
        
        # LSTM Layers
        self.lstm_1 = nn.LSTM(input_size, hidden_sizes[0], num_layers=3,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_21 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[1], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_22 = nn.LSTM(input_size, hidden_sizes[1], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_31 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[2], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_32 = nn.LSTM(4*hidden_sizes[1], hidden_sizes[2], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_41 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[3], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_42 = nn.LSTM(4*hidden_sizes[2], hidden_sizes[3], num_layers=3,
                             batch_first=True, bidirectional=True, dropout=dropout)
        hidd = 2*hidden_sizes[0] + 4*(hidden_sizes[1]+hidden_sizes[2]+hidden_sizes[3])
        self.lstm_5 = nn.LSTM(hidd, hidden_sizes[4], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Sequential(nn.Linear(576, 4096),
                                nn.ReLU(),
                                nn.Dropout(p=dropout),
                                nn.Linear(4096, 1024),
                                nn.ReLU(),
                                nn.Dropout(p=dropout),
                                nn.Linear(1024, output_size),
                               )
        
    def forward(self, x):
        # lstm layers:
        x1, _ = self.lstm_1(x)
        
        x_x1, _ = self.lstm_21(x1)
        x_x2, _ = self.lstm_22(x)
        x2 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_31(x_x1)
        x_x2, _ = self.lstm_32(x2)
        x3 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_41(x_x1)
        x_x2, _ = self.lstm_42(x3)
        x4 = torch.cat([x_x1, x_x2], dim=2)
        x = torch.cat([x1, x2, x3, x4], dim=2)
        x, _ = self.lstm_5(x)
        
        # fully connected layers:
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
### VALIDATION FUNCTION
def validation(model, loader, criterion, device="cpu"):
    model.eval()
    loss = 0
    preds_all = torch.LongTensor()
    labels_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x, labels in loader:
            labels_all = torch.cat((labels_all, labels), dim=0)
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            output = model.forward(batch_x)
            loss += criterion(output,labels).item()
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
    total_loss = loss/len(loader)
    
    return total_loss

In [ ]:
### TRAINING FUNCTION
def train_model(model, trainloader, validloader, criterion, optimizer, 
                scheduler, epochs=20, device="cpu", print_every=1):
    model.to(device)
    best_auc = 0
    best_epoch = 0
    for e in range(epochs):
        model.train()
        
        for batch_x, labels in trainloader:
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            # Training 
            optimizer.zero_grad()
            output = model.forward(batch_x)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
        # at the end of each epoch calculate loss and auc score:
        model.eval()
        train_loss = validation(model, trainloader, criterion, device)
        valid_loss = validation(model, validloader, criterion, device)
        if e % print_every == 0:
            to_print = "Epoch: "+str(e+1)+" of "+str(epochs)
            to_print += ".. Train Loss: {:.4f}".format(train_loss)
            to_print += ".. Valid Loss: {:.4f}".format(valid_loss)

            print(to_print)
    # After Training:
    torch.save(model.state_dict(), "best-state.pt")
    model.load_state_dict(torch.load("best-state.pt"))
    to_print = "\nTraining completed. Best state dict is loaded.\n"
    print(to_print)
    return

In [ ]:
model_lstm.load_state_dict(torch.load("best-state.pt"))
def prediction(model, loader, device="cpu"):
    model.to(device)
    model.eval()
    preds_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x in loader:
            batch_x = batch_x.to(device)
            
            output = model.forward(batch_x).to("cpu")
            preds_all = torch.cat((preds_all, output), dim=0)
    return preds_all

In [ ]:
sequence_length = len(x_train_t)
input_size = len(d_list[1][1])
hidden_sizes = [288, 192, 144, 96, 32]
max_learning_rate = 0.0001
epochs = 4000

# Model
model_lstm = RNN(input_size, hidden_sizes, sequence_length)
print("Model: ")
print(model_lstm)

# criterion, optimizer, scheduler
criterion = nn.MSELoss()
#criterion = nn.BCELoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=max_learning_rate)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                          max_lr = max_learning_rate,
                                          epochs = epochs + 1,
                                          steps_per_epoch = int(len(dataloaders["train"])),
                                          pct_start = 0.2,
                                          anneal_strategy = "cos")

In [ ]:
%%time
# Checking if GPU is available
if torch.cuda.is_available():
    my_device = "cpu"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

# Training
train_model(model = model_lstm,
            trainloader = dataloaders["train"],
            validloader = dataloaders["val"],
            criterion = criterion,
            optimizer = optimizer,
            scheduler = scheduler,
            epochs = epochs,
            device = my_device,
            print_every = 2)

In [ ]:
y_test[:]

In [ ]:
if torch.cuda.is_available():
    my_device = "cpu"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

In [ ]:
y_train_t

In [ ]:
y_pred = prediction(model_lstm, dataloaders["test2"], device=my_device)
print("Prediction completed, first 5 states:")
y_pred[:]